# Validacion Cruzada K-Fold

Es una tecnica robusta para evaluar el rendimiento de un modelo de Machine Learning y evitar problemas como el sobreajuste (*overfitting*). El proceso general de este metodo es el siguiente:

1.- **Dividir de los datos**: El conjunto de datos se divide en $K$ subconjuntos aproximadamente del mismo tamano. A estos se les conoce como "*Fold*"

2.- **Iterar a traves de los Folds**: Se realizan $K$ iteraciones de entrenamiento y prueba. En cada iteracion, unos de los K folds se utiliza como conjunto de prueba y los otros $K-1$ folds como conjunto de entrenamiento. Esto implica que cada observacion se utiliza exactamente una vez como conjunto de prueba y $K-1$ veces como parte del conjunto de entrenamiento.

3.- **Evaluar el modelo**: Se obtienen las metricas del modelo para cada iteracion de los folds. 

4.- **Promediar resultados**: Despues de las $K$ iteraciones, se promedian las metricas de rendimiento (como precision, MSE, R-cuadrado, entre otros) para obtener una estimacion mas confiable del rendimiento del modelo

## Ventajas y Desventajas

- Al utilizar todos los datos tanto para entrenamiento como para prueba, se obtiene una mejor estimacion del rendimiento del modelo.

- Al promediar los resultados de las iteraciones, se reduce la varianza asociada con cualquier particion de los datos.

- A diferencia de la validacion Hold Out, se utilizan todos los datos disponibles tanto para entrenamiento como para prueba

- Su principal desventaja es que requiere entrena el modelo $K$ veces lo que implica un alto costo computacional especialmente para modelos complejos y grandes conjuntos de datos.

## Implementacion Manual

Vamos a implementar la Validacion Cruzada K-Fold de forma manual para entender paso a paso, en que consiste este metodo. Para iniciar, creamos un conjunto de datos aleatorio a los cuales aplicaremos la validacion 

In [4]:
# librerias de manejo de datos
import numpy as np
import pandas as pd

# semilla aleatoria
np.random.seed(42)

# 3 Variables predictoras
X = pd.DataFrame({
    'var_1': np.random.rand(100),
    'var_2': np.random.rand(100),
    'var_3': np.random.rand(100),
})

# Variable objetivo
y = 3*X['var_1'] + 2*X['var_3'] + np.random.randn(100)

# y convertida en Serie
df_y = pd.Series(y, name='y')

# concatenamos la variable objetivo con las predictoras
df = pd.concat([X, df_y], axis=1)

# mostrar datos creados
df


,var_1,var_2,var_3,y
0,0.374540,0.031429,0.642032,2.453255
1,0.950714,0.636410,0.084140,2.368823
2,0.731994,0.314356,0.161629,4.663183
3,0.598658,0.508571,0.898554,4.227003
4,0.156019,0.907566,0.606429,-0.344229
...,...,...,...,...
95,0.493796,0.349210,0.522243,3.317536
96,0.522733,0.725956,0.769994,3.732305
97,0.427541,0.897110,0.215821,2.342611
98,0.025419,0.887086,0.622890,1.309792


Se han creado 100 observaciones con datos aleatorios para 3 variables predictoras y una objetivo. Decidimos que la variable 'y' presente una relacion lineal con las variables var_1 y var_3 para tener certeza sobre los resultados esperados.

###  **1.- Dividir los datos**

Definimos el numero de folds que popularmente se toma $K=5$ o $K=10$ y calculamos el tamano de cada fold dividiendo el numero total de observaciones entre $K$

In [5]:
# numero de folds
K = 5
# numero total de muestras (100 en este caso)
n_muestras = len(X)
# tamano de los folds
size_fold = n_muestras // K
# mostrar tamano de los folds
size_fold

20

Tenemos 5 folds de 20 observaciones cada uno. Ahora, mezclamos los indices de las observaciones aleatoriamente para asegurar que la division de los folds sea aleatoria

In [7]:
# mezcla aleatoria de los indices
mix_index = np.random.permutation(n_muestras)
# reordenar valores de X con indices mezclados
X_mix = X.iloc[mix_index]
# reordenar valores de y con indices mezclados
y_mix = y.iloc[mix_index]
# mostrar Variables predicotras mezcladas
X_mix

,var_1,var_2,var_3
90,0.119594,0.093103,0.030500
40,0.122038,0.962447,0.940459
87,0.637557,0.555201,0.542645
19,0.291229,0.539342,0.849223
88,0.887213,0.529651,0.286541
...,...,...,...
16,0.304242,0.803672,0.325400
62,0.828738,0.633530,0.140084
26,0.199674,0.818015,0.973011
63,0.356753,0.535775,0.518330


Una vez mezclada las observaciones, inicializamos las listas que almacenaran las metricas del modelo. En este caso utilizaremos un modelo de Regresion Lineal por lo que las metricas a evaluar son $R^2$ y $MSE$. Debemos crear un par de listas para las metricas del conjunto entrenamiento y otro par para el conjunto de prueba en cada fold.

In [10]:
# librerias para usar el modelo y las metricas
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# listas para metricas de entrenamiento
train_r2 = []
train_mse = []

# listas para metricas de prueba
test_r2 = []
test_mse = []

### **2 y 3.- Iterar sobre cada fold y entrenar el modelo K veces**

Con un bucle, iteramos sobre cada fold con el que se entrena el modelo y se calcula las metricas tanto para el conjunto de entrenamiento como de prueba.

In [12]:
for k in range(K):
    # indice inicial fold actual
    ini = k*size_fold
    # indice final fold actual
    fin = (k+1)*size_fold
    
    #asignar observaciones al conjunto de prueba
    X_test = X_mix.iloc[ini:fin]
    y_test = y_mix.iloc[ini:fin]
    # asignar las observaciones restantes al conjunto de entrenamiento
    X_train = pd.concat([X_mix.iloc[:ini], X_mix.iloc[fin:]])
    y_train = pd.concat([y_mix.iloc[:ini], y_mix.iloc[fin:]])
    
    # instanciar y entrenar el modelo
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # predicciones con datos de entrenamiento
    y_train_pred = model.predict(X_train)
    # predicciones con datos de prueba
    y_test_pred = model.predict(X_test)
    
    # calculo de metricas de entrenamiento
    train_r2.append(r2_score(y_train, y_train_pred))
    train_mse.append(mean_squared_error(y_train, y_train_pred))
    
    # calculo de metricas de prueba
    test_r2.append(r2_score(y_test, y_test_pred))
    test_mse.append(mean_squared_error(y_test, y_test_pred))
    

### **4.- Promediar resultados**

Se promedian las metricas para los conjuntos de entrenamiento y prueba a lo largo de todos los folds.

In [17]:
print(f'Promedio de R-cuadrado para datos de entrenamiento: {round(np.mean(train_r2),4)}')
print(f'Promedio de R-cuadrado para datos de prueba: {round(np.mean(test_r2),4)}')
print(f'Promedio de MSE para datos de entrenamiento: {round(np.mean(train_mse),4)}')
print(f'Promedio de MSE para datos de prueba: {round(np.mean(test_mse),4)}')

Promedio de R-cuadrado para datos de entrenamiento: 0.6075
Promedio de R-cuadrado para datos de prueba: 0.5688
Promedio de MSE para datos de entrenamiento: 0.9214
Promedio de MSE para datos de prueba: 0.9932


> La diferencia entre el R² y el MSE en los conjuntos de entrenamiento y prueba sugiere que el modelo puede estar sobreajustado. Está capturando bien la variabilidad en los datos de entrenamiento, pero su capacidad para generalizar a datos nuevos es algo limitada ($R^2$ de entrenamiento es mayor que $R^2$ de prueba).

> El sobreajuste ocurre cuando un modelo se ajusta demasiado bien a los datos de entrenamiento y no generaliza bien a nuevos datos. En tu caso, el R² es más alto y el MSE es más bajo en el conjunto de entrenamiento en comparación con el conjunto de prueba.